In [1]:
# https://www.nih.gov/news-events/news-releases/nih-clinical-center-provides-one-largest-publicly-available-chest-x-ray-datasets-scientific-community

import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms as tf
from os import walk
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import image
from torchvision import transforms
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
train_df = pd.read_csv('train_df2.csv')
test_df = pd.read_csv('test_df2.csv')

In [3]:
train_df.sample(10)

,Finding Label,Image Index,h,w,x,y
84989,4,00029546_000.png,2021.0,2021.0,0.194311,0.194311
41992,1,00012927_011.png,2048.0,2500.0,0.168000,0.168000
71536,8,00022569_001.png,2544.0,3056.0,0.139000,0.139000
74872,1,00024656_000.png,2544.0,3056.0,0.139000,0.139000
74194,1,00024048_000.png,2500.0,2048.0,0.168000,0.168000
49662,8,00015355_013.png,2991.0,2992.0,0.143000,0.143000
83234,4,00028356_000.png,3056.0,2544.0,0.139000,0.139000
61098,13,00018964_014.png,2048.0,2500.0,0.168000,0.168000
73952,1,00023811_000.png,2991.0,2992.0,0.143000,0.143000
25860,13,00008042_004.png,2048.0,2500.0,0.168000,0.168000


In [4]:
train_df['Finding Label'].value_counts()

1     50500
4      8758
8      8280
5      5361
6      2429
3      2198
12     2056
0      1515
9      1241
13     1122
7       969
11      902
10      868
14      235
2        90
Name: Finding Label, dtype: int64

In [5]:
# (train_df['Image Index'] == '00030181_001.png').value_counts()
# 00028247_001.png

In [6]:
import os
import platform
my_path = "../Datasets/Lungs_Dataset/Xray" if platform.system() == 'Windows' else "datasets/data/images"
filename_list = []
for root, dirs, files in os.walk(my_path, topdown=True):
    for name in files:
        filename_list.append(name)
#         with Image.open(os.path.join("datasets/data/images", name)) as f:
#             print(len(f.size))

In [7]:
train_df['Image Index'].isin(filename_list).value_counts()

True    86524
Name: Image Index, dtype: int64

In [8]:
train_df = train_df[train_df['Image Index'].isin(filename_list)]

In [9]:
train_df['Image Index'].isin(filename_list).value_counts()

True    86524
Name: Image Index, dtype: int64

In [10]:
train_df['Image Index'].isna().value_counts()

False    86524
Name: Image Index, dtype: int64

In [11]:
test_df = test_df[test_df['Image Index'].isin(filename_list)]

In [12]:
test_df.sample(10)

,Finding Label,Image Index,h,w,x,y
4891,8,00007886_021.png,2048.0,2500.0,0.168,0.168
18710,1,00021926_001.png,2163.0,2310.0,0.143,0.143
10008,8,00013625_052.png,2048.0,2500.0,0.168,0.168
12109,4,00014956_042.png,2048.0,2500.0,0.168,0.168
8913,1,00012636_004.png,2991.0,2992.0,0.143,0.143
21935,1,00028112_000.png,2991.0,2570.0,0.143,0.143
7551,13,00011391_044.png,2048.0,2500.0,0.168,0.168
5740,1,00009368_020.png,2048.0,2500.0,0.168,0.168
12977,8,00015831_009.png,2991.0,2992.0,0.143,0.143
22829,8,00028774_048.png,3056.0,2544.0,0.139,0.139


In [13]:
class RescaleImage(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
            
        The samples coming into this class will have its images reduced assuming
        the input is a h, w, c numpy array
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        h, w = image.size

        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = image.resize((new_h, new_w))

        return {'image': img, 'label': label}

In [14]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = np.array(image)
        if len(image.shape) > 2:
            image = image[:,:,0]
        image = torch.FloatTensor(image).unsqueeze(0)
        label = torch.from_numpy(np.array(label))
        # numpy image: H x W x C
        # torch image: C X H X W
#         image.to(device)
#         label.to(device)
        return {'image': image,
#         return {'image': torch.from_numpy(image),                
                'label': label}

In [15]:
#  print([Image.open('../Datasets/Lungs_Dataset/Xray/'+train_df['Image Index'][each_import]).size for each_import in range(100)])

In [16]:
#Note this will return an Image object, of h and w, and its corresponding label
class CovidLungsDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, dataframe, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.dataframe['Image Index'][idx])
        my_image = Image.open(img_name)        
        if len(my_image.size) > 2:
            assert len(my_image.size) > 2
        row = self.dataframe.iloc[idx]
        label = row['Finding Label']
        sample = {'image': my_image, 'label': label}
        if self.transform:
            sample = self.transform(sample)

        return sample

In [17]:
my_train_set = CovidLungsDataset(train_df, my_path, transform=transforms.Compose([
        RescaleImage(200),
        ToTensor()
]))

In [18]:
'''The original data coming out of the Dataset is a dictionary, 
the image is an Image object with the corresponding label.
The size of all of these images are 1024 by 1024.
The RescaleImage will convert the Image object to an Image object with 200x200 in size, 
leaving the label alone.
The ToTensor will convert the Image object with 200x200 to tensors of 1x200x200'''
from random import randint
a_sample = my_train_set.__getitem__(randint(0,256))
my_image = a_sample['image']
my_label = a_sample['label']
#imshow will work if it's h,w,c or h,w
#torch is c,h,w
#line below can be used if you don't use ToTensor
# plt.imshow(my_image)
type(my_image), my_image.shape, my_label

(torch.Tensor, torch.Size([1, 200, 200]), tensor(1))

In [19]:
# plt.imshow(my_image.squeeze(0));

In [20]:
import platform
batch_loader_params = {
    "batch_size": 25,
    "shuffle": True,
    "num_workers": 0 if platform.system() == 'Windows' else 2
#     "num_workers": 2
}
dataloader = DataLoader(my_train_set, **batch_loader_params)

In [21]:
# iter(dataloader).next()
# for i, each in enumerate(dataloader):
#     print(each['image'].shape, each['label'].shape)

In [22]:
# batch_samples = iter(dataloader)
# samples = batch_samples.next()
# datset_batch = torchvision.utils.make_grid(samples['image'])

In [23]:
# plt.figure(figsize=(20,10))
# for index, each in enumerate(datset_batch):
#     plt.imshow(each.squeeze(0))

In [24]:
def spatial_size(input_size: int, kernel_size: int, stride: int = 1, padding: int = 0):
    # https://cs231n.github.io/convolutional-networks/
    spatial_size = (input_size - kernel_size + 2 * padding)/stride + 1
    assert spatial_size % 1 == 0
    assert spatial_size > 0
    return int(spatial_size)

In [25]:
print(spatial_size(200, 51))

150


In [26]:
print(spatial_size(75, 50))

26


In [27]:
print(spatial_size(13, 8))

6


In [28]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 200, 51, 1)
        self.conv2 = nn.Conv2d(200, 75, 50, 1)        
        self.conv3 = nn.Conv2d(75, 13, 8, 1)  
        self.fc1 = nn.Linear(3*3*13, 32)
        self.fc2 = nn.Linear(32, 15)

    def forward(self, x):
        x = F.relu(self.conv1(x)) # torch.Size([5, 1, 200, 200]) ==> torch.Size([5, 200, 150, 150])
        x = F.max_pool2d(x, 2, 2) # torch.Size([5, 200, 150, 150]) ==> torch.Size([5, 200, 75, 75])
        x = F.relu(self.conv2(x)) # torch.Size([5, 200, 75, 75]) ==> torch.Size([5, 75, 26, 26])
        x = F.max_pool2d(x, 2, 2) # torch.Size([5, 75, 26, 26]) ==> torch.Size([5, 75, 13, 13])
        x = F.relu(self.conv3(x)) # torch.Size([5, 75, 13, 13]) ==> torch.Size([5, 13, 6, 6])
        x = F.max_pool2d(x, 2, 2) # torch.Size([5, 13, 6, 6]) ==> torch.Size([5, 13, 3, 3])
        x = x.view(-1, 3*3*13)  # torch.Size([5, 13, 3, 3]) ==> torch.Size([5*32, 3*3*13])
        x = F.relu(self.fc1(x))   # torch.Size([5*32, 3*3*13]) ==> torch.Size([5, 32])
        x = self.fc2(x)           # torch.Size([5, 32]) ==> torch.Size([5, 15])
        # # There's no activation at the final layer because of the criterion of CEL
#         return x
        return torch.log_softmax(x, dim=-1)


net = Net()
net = nn.DataParallel(net)
net.to(device)
print(net)

DataParallel(
  (module): Net(
    (conv1): Conv2d(1, 200, kernel_size=(51, 51), stride=(1, 1))
    (conv2): Conv2d(200, 75, kernel_size=(50, 50), stride=(1, 1))
    (conv3): Conv2d(75, 13, kernel_size=(8, 8), stride=(1, 1))
    (fc1): Linear(in_features=117, out_features=32, bias=True)
    (fc2): Linear(in_features=32, out_features=15, bias=True)
  )
)


In [29]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [ ]:
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data['image'], data['label']
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        image = data['image']
        image = image.to(device)
        outputs = net(image)
        label = data['label']
        label = label.to(device)
#         print(outputs.shape, data['label'].shape)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        print('[%d, %5d] loss: %.5f' %
              (epoch + 1, i + 1, running_loss / (epoch*i+1)))

[1,     1] loss: 4.18492
[1,     2] loss: 7567022.18492
[1,     3] loss: 7605206.67320
[1,     4] loss: 7881983.29820
[1,     5] loss: 7912978.20640
[1,     6] loss: 7917495.73130
[1,     7] loss: 7917654.59980
[1,     8] loss: 7917671.60826
[1,     9] loss: 7917700.73724
[1,    10] loss: 7917704.30781
[1,    11] loss: 7917714.62088
[1,    12] loss: 7917717.09748
[1,    13] loss: 7917783.44069
[1,    14] loss: 7917791.46053
[1,    15] loss: 7917794.53128
[1,    16] loss: 7917797.57003
[1,    17] loss: 7917800.39177
[1,    18] loss: 7917803.26041
[1,    19] loss: 7917805.72489
[1,    20] loss: 7917808.27452
[1,    21] loss: 7917810.82823
[1,    22] loss: 7917813.40336
[1,    23] loss: 7917815.94667
[1,    24] loss: 7917818.48659
[1,    25] loss: 7917820.77073
[1,    26] loss: 7917823.21249
[1,    27] loss: 7917836.12751
[1,    28] loss: 7921652.91877
[1,    29] loss: 7921655.10867
[1,    30] loss: 7921657.41985
[1,    31] loss: 7921659.69156
[1,    32] loss: 7921661.91714
[1,    33] los

[1,   265] loss: 7932953.42437
[1,   266] loss: 7932954.91249
[1,   267] loss: 7932956.55210
[1,   268] loss: 7932957.91259
[1,   269] loss: 7932959.36167
[1,   270] loss: 7932960.76396
[1,   271] loss: 7932962.45291
[1,   272] loss: 7932964.10769
[1,   273] loss: 7932965.61680
[1,   274] loss: 7932967.55392
[1,   275] loss: 7932970.19170
[1,   276] loss: 7932972.07306
[1,   277] loss: 7932973.64638
[1,   278] loss: 7932975.32546
[1,   279] loss: 7932976.90977
[1,   280] loss: 7932978.57457
[1,   281] loss: 7932980.67362
[1,   282] loss: 7932982.00315
[1,   283] loss: 7932983.33446
[1,   284] loss: 7932984.71017
[1,   285] loss: 7932986.31126
[1,   286] loss: 7932987.73280
[1,   287] loss: 7932989.44333
[1,   288] loss: 7932990.99986
[1,   289] loss: 7932992.14540
[1,   290] loss: 7932994.13407
[1,   291] loss: 7932995.04917
[1,   292] loss: 7932996.66573
[1,   293] loss: 7932997.52645
[1,   294] loss: 7932999.39548
[1,   295] loss: 7933000.83050
[1,   296] loss: 7933003.44695
[1,   29

In [ ]:
torch.cuda.empty_cache()